In [1]:
%load_ext autoreload
%autoreload 2
import os; import sys; sys.path.insert(0,'../')
import pandas as pd
from tqdm.auto import tqdm
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [2]:
import sys
sys.path.insert(0, '../../')

import atomic.features as fs
import atomic.labels as lab

In [3]:
## Configure file and folder names
datafolder = "../data/socceraction/"
spadl_h5 = os.path.join(datafolder,"spadl-statsbomb.h5")
features_h5 = os.path.join(datafolder,"atomic-features.h5")
labels_h5 = os.path.join(datafolder,"atomic-labels.h5")
predictions_h5 = os.path.join(datafolder,"atomic-predictions.h5")

In [4]:
games = pd.read_hdf(spadl_h5,"games")

print("nb of games:", len(games))

actiontypes = pd.read_hdf(spadl_h5, "atomic_actiontypes")
bodyparts = pd.read_hdf(spadl_h5, "bodyparts")

nb of games: 452


In [5]:
xfns = [fs.actiontype,
       fs.actiontype_onehot,
       fs.bodypart,
       fs.bodypart_onehot,
       fs.goalscore,
       fs.location,
       fs.polar,
       fs.direction,
       fs.team,
       fs.time,
       fs.time_delta
      ]

with pd.HDFStore(spadl_h5) as spadlstore, pd.HDFStore(features_h5) as featurestore:
    for game in tqdm(list(games.itertuples()),desc=f"Generating and storing features in {features_h5}"):
        actions = spadlstore[f"atomic_actions/game_{game.game_id}"]
        actions = (
            actions.merge(actiontypes,how="left")
            .merge(bodyparts,how="left")
            .reset_index(drop=True)
        )
        gamestates = fs.gamestates(actions, 3) # Taking 3 previous actions
        gamestates = fs.play_left_to_right(gamestates,game.home_team_id)

        X = pd.concat([fn(gamestates) for fn in xfns],axis=1)
        featurestore[f"game_{game.game_id}"] = X

In [6]:
yfns = [lab.scores,lab.concedes,lab.goal_from_shot]

with pd.HDFStore(spadl_h5) as spadlstore, pd.HDFStore(labels_h5) as labelstore:
    for game in tqdm(list(games.itertuples()),desc=f"Computing and storing labels in {labels_h5}"):
        actions = spadlstore[f"atomic_actions/game_{game.game_id}"]
        actions = (
            actions.merge(actiontypes,how="left")
            .merge(bodyparts,how="left")
            .reset_index(drop=True)
        )
        Y = pd.concat([fn(actions) for fn in yfns],axis=1)
        labelstore[f"game_{game.game_id}"] = Y